# Corretor de Redações Dissertativa-Argumentativas

![](https://ogimg.infoglobo.com.br/brasil/educacao/enem-e-vestibular/25245061-ea2-64a/FT1086A/redacao-enem.jpg)

## Descrição Geral

Projeto realizado na disciplina Processamento de Linguagem Natural (PLN) do curso de Ciência da Computação da UFCG.

## Equipe
* Ingrid Jackeline dos Santos Castro
* Vinicius Ferreira de Sousa

## Objetivo

Este projeto visa utilizar o modelo Llama 3.1 Instruct, com 8 bilhões de parâmetros, para realizar correções de textos dissertativo-argumentativos (estilo ENEM), fornecendo feedbacks detalhados e construtivos, com sugestões de melhorias, acerca das redações providas pelos usuários.

## Metodologia

A metodologia proposta consiste em realizar a técnica de Fine-tuning do Llama 3.1 Instruct (8B), através da bilioteca Unsloth, que viabiliza a execução otimizada e eficiente dos modelos open source disponíveis no mercado, aliada à técnica de Prompt Engineering, instruindo o modelo a se comportar como um corretor de redações.

## Fine-tuning do Modelo Llama 3.1 Instruct (8B)

### Imports

Imports das ferramentas necessárias para a manipulação do Dataset e Treinamento do Modelo.

In [1]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [3]:
import pandas as pd
import torch

from datasets import Dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported

### Carregamento do Dataset

O dataset utilizado foi construído com textos de 15 redações reais, disponíveis no [Banco de Redações do UOL](https://educacao.uol.com.br/bancoderedacoes/propostas/qualificacao-e-o-futuro-do-emprego.htm), sendo elas as 7 primeiras e as 3 últimas. As 5 restantes foram extraídas dos materiais de um Cursinho Preparatório para o Enem, fornecido por um professor do IFPB-Esperança.
Além disso, ele também inclui as respectivas correções de cada redação, as quais foram geradas pelo ChatGPT mediante o prompt de sistema que será utilizado a seguir para o treinamento do modelo.

In [13]:
filepath = 'https://drive.usercontent.google.com/download?id=1J4JnBurNlXf6GlK4fTNmeiDtxSn7Q2sD&export=download'
data_df = pd.read_csv(filepath)

data_df

,redacao,feedback
0,Tema: Qualificação e o futuro do emprego\nTítu...,**Visão Geral:**\nO texto aborda a evolução da...
1,Tema: Qualificação e o futuro do emprego\nTítu...,**Visão geral** \nO texto aborda o tema da qu...
2,Tema: Qualificação e o futuro do emprego\nTítu...,**Visão geral:** \nO texto aborda a problemát...
3,Tema: Qualificação e o futuro do emprego\nTítu...,"Aqui está a análise detalhada da redação, conf..."
4,Tema: Qualificação e o futuro do emprego\nTítu...,### Visão geral\nO texto discute a relação ent...
5,Tema: Qualificação e o futuro do emprego\nTítu...,Aqui está a análise detalhada da sua redação d...
6,Tema: Qualificação e o futuro do emprego\nTítu...,Aqui está o feedback detalhado e construtivo p...
7,"A Constituição Federal de 1988, norma de maior...",Visão geral\nO texto está inserido no domínio ...
8,O avanço tecnológico na internet deveria ser i...,Visão geral\nO texto aborda a manipulação de c...
9,A democratização do acesso ao cinema deveria c...,Visão geral\nO texto trata da falta de democra...


In [14]:
sys_prompt = """Seu papel é ser um corretor de redações dissertativa-argumentativas, fornecendo feedbacks detalhados e construtivos a fim de melhorar significativamente a qualidade dos textos recebidos. 
Siga as instruções abaixo rigorosamente, gerando respostas para cada um dos tópicos delimitados pelo caractere #:

# Visão geral
Apresente uma visão geral e objetiva do texto, descrevendo sobre o domínio no qual ele está inserido.

# Ortografia
Aponte e corrija os erros ortográficos de forma direta, se houver. Dê exemplos dos erros e indique a correção, sem reescrever o trecho completo.

# Estrutura
Identifique as seguintes partes que compõem o texto, especificadas pelos itens 1, 2 e 3 abaixo:
1 - Introdução: Apresentação e contextualização do tema. Situa-se no primeiro parágrafo.
2 - Desenvolvimento: Construção de argumentos que defendem um determinado ponto de vista. Situa-se nos parágrafos intermediários.
3 - Conclusão: Apresentação de propostas de intervenção e agentes que podem resolver o problema. Situa-se no último parágrafo.
Discurse sobre cada uma das partes de forma objetiva, levantando pontos que podem ser melhorados, sem reescrever trechos.

# Competências do ENEM
Analise o texto conforme as 5 competências do ENEM, especificadas pelos itens de 1 a 5 a seguir, e forneça um feedback para cada uma delas:
1 - Domínio da escrita formal da língua portuguesa.
2 - Compreensão do tema e aplicação das áreas de conhecimento.
3 - Selecionar, relacionar, organizar e interpretar informações, fatos, opiniões e argumentos em defesa de um ponto de vista.
4 - Domínio dos mecanismos linguísticos de argumentação.
5 - Elaboração de proposta uma de intervenção, respeitando os direitos humanos.

Não atribua pontuações a nenhum dos tópicos detalhados acima. Seu principal objetivo é orientar o usuário a melhorar a qualidade das redações escritas, seguindo as diretrizes mencionadas."""

A função `format_train_prompt` formata a sequência de prompts a serem passados para o modelo durante o treinamento, adicionando tags que delimitam os prompts de sistema, user e assistant dentro do contexto de correção das redações.

In [15]:
def format_train_prompt(system_prompt, user_prompt, assistant_prompt):
    return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>{sys_prompt}<|eot_id|>
    <|start_header_id|>user<|end_header_id|>{user_prompt}<|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>{assistant_prompt}<|eot_id|>"""

A função `prepare_train_data` adiciona uma nova coluna ao Dataset, que contém a sequência de prompts e os marcadores necessários para o treinamento do modelo, convertendo os textos para um formato processável.

In [16]:
def prepare_train_data(data):
    data["text"] = data[["redacao", "feedback"]].apply(lambda prompt: format_train_prompt(sys_prompt, prompt["redacao"], prompt["feedback"]), axis=1)
    data = Dataset.from_pandas(data)
    
    return data

In [17]:
data = prepare_train_data(data_df)

Recorte do Dataset com a nova coluna `text`.

In [18]:
data_df.head()

,redacao,feedback,text
0,Tema: Qualificação e o futuro do emprego\nTítu...,**Visão Geral:**\nO texto aborda a evolução da...,<|begin_of_text|><|start_header_id|>system<|en...
1,Tema: Qualificação e o futuro do emprego\nTítu...,**Visão geral** \nO texto aborda o tema da qu...,<|begin_of_text|><|start_header_id|>system<|en...
2,Tema: Qualificação e o futuro do emprego\nTítu...,**Visão geral:** \nO texto aborda a problemát...,<|begin_of_text|><|start_header_id|>system<|en...
3,Tema: Qualificação e o futuro do emprego\nTítu...,"Aqui está a análise detalhada da redação, conf...",<|begin_of_text|><|start_header_id|>system<|en...
4,Tema: Qualificação e o futuro do emprego\nTítu...,### Visão geral\nO texto discute a relação ent...,<|begin_of_text|><|start_header_id|>system<|en...


### Carregamento do Modelo

A versão Instruct do Llama 3.1, com 8 bilhões de parâmetros, foi utilizada em virtude da sua capacidade de obedecer instruções e fornecer respostas. Outras versões apenas tentam completar o texto passado como input, o que acaba dificultando a realização do Prompt Engineering.
Além disso, também foi empregada a versão quantizada com 4 bits, que torna o modelo mais leve, com um menor consumo de memória, e viabiliza o treinamento em condições computacionalmente limitadas.

In [20]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length=2048,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.888 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 6.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Utilização da técnica `PEFT` (Parameter-Efficient Fine-Tuning), que permite ajustar apenas partes específicas do modelo ao invés de todo o conjunto de parâmetros, economizando recursos computacionais:

* **r=16:** rank de LoRA (Low-Rank Adaptation), que ajusta os parâmetros em um espaço de baixa dimensão
* **target_modules:** ajuste dos projetores de atenção (q_proj, k_proj, etc.)
* **lora_alpha=16:** fator de escala que controla o impacto das atualizações LoRA no treinamento
* **lora_dropout=0:** não há dropout aplicado às camadas de LoRA
* **bias="none":** não há viés adicional a ser aplicado
* **use_gradient_checkpointing="unsloth":** estratégia de checkpointing de gradientes para economizar memória
* **random_state=3407:** semente usada para garantir a reprodutibilidade do treinamento
* **use_rslora=False:** sem utilização da técnica rslora
* **loftq_config=None:** sem configuração específica de quantização

In [21]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Treinamento

Os argumentos de treinamento aplicados foram os seguintes:

* **per_device_train_batch_size=2:** tamanho do batch por dispositivo (GPU ou CPU)
* **gradient_accumulation_steps=4:** acumulação dos gradientes por 4 passos antes de realizar uma atualização dos pesos, o que permite trabalhar com batches maiores sem estourar a memória da GPU
* **num_train_epochs=4:** número de épocas de treinamento
* **warmup_steps=5:** número de passos de warmup para o agendador de taxa de aprendizado
* **learning_rate=2e-4:** taxa de aprendizado inicial
* **fp16:** usa a precisão de 16 bits para economizar memória
* **logging_steps=1:** log das informações a cada 1 passo de treinamento
* **optim="adamw_8bit":** otimizador AdamW com representação de 8 bits para economizar memória
* **weight_decay=0.01:** decaimento de peso para regularização
* **lr_scheduler_type="linear":** agendador linear para a taxa de aprendizado
* **seed=3407:** semente para reprodutibilidade
* **output_dir="outputs":** diretório de saída para salvar os resultados
* **report_to="none":** sem report a ferramentas externas

In [22]:
train_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    num_train_epochs = 4,
    warmup_steps = 5,
    learning_rate = 2e-4,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = "outputs",
    report_to = "none"
    )

Os parâmetros utilizados para o `SFTTrainer` (Supervised Fine-tuning Trainer) foram os seguintes:

* **model=model:** o modelo
* **tokenizer=tokenizer:** tokenizador utilizado para processar os dados de entrada
* **train_dataset=data:** dataset de treinamento, que contém o texto a ser usado
* **dataset_text_field="text":** campo do dataset onde o texto está localizado
* **max_seq_length=2048:** tamanho máximo das sequências de entrada (2048 tokens)
* **dataset_num_proc=2:** dois processos para paralelizar o carregamento do dataset
* **args=train_args:** argumentos de treinamento definidos anteriormente

In [23]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = data,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    args = train_args
)

Map (num_proc=2):   0%|          | 0/15 [00:00<?, ? examples/s]

In [24]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 15 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 8
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.566900
2,1.549400
3,1.531400
4,1.515000
5,1.490500
6,1.370100
7,1.354300
8,1.245700


TrainOutput(global_step=8, training_loss=1.4529308080673218, metrics={'train_runtime': 841.7772, 'train_samples_per_second': 0.071, 'train_steps_per_second': 0.01, 'total_flos': 5556864352493568.0, 'train_loss': 1.4529308080673218, 'epoch': 4.0})

### Inferência

Correções geradas pelo modelo para vários exemplos de redações.

In [46]:
model = FastLanguageModel.for_inference(model)

Redação com temática distinta das abordadas pelas redações utilizadas no treinamento.

In [47]:
prompt_test = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>{sys_prompt}<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Tema: A estética corporal como integradora social e formadora de identidade
A imposição de padrões midiáticos e a obsessão por verdades absolutas sobre o “belo” e o
“feio”, atrelados a uma educação pautada na competitividade, criam mitos relacionados ao corpo
e limitam a convivência com a diversidade de modelos corporais. Com isso, surgem preconceitos e
estereótipos, que devem ser combatidos.
É válido pontuar que, no Brasil, a população foi formada de maneira miscigenada, com
isso, surgiram diversos modelos corporais como, por exemplo, pessoas brancas, negras, altas, baixas,
gordas ou magras. Contudo, por causa da imposição de padrões de beleza, feita através da mídia,
essa diversidade de formas corporais é, muitas vezes, esquecida em detrimento do que é valorizado
nas propagandas, como mulheres magras e homens musculosos. Assim, criam-se mitos sobre pa-
drões de corpos ideais, os quais se refletem em uma cultura que classifica as pessoas pela identidade
corporal, como pensa Miriam Goldemberg.
17Como consequência, as pessoas que não se enquadram nos padrões impostos pela mídia,
sejam elas gordas ou magras demais, são, muitas vezes, vítimas de estereótipos e preconceito como,
por exemplo, a associação da obesidade ao descaso ou ao fracasso. Isso reflete um ensino brasileiro
pautado, principalmente, na competitividade, que forma pessoas individualistas e ignorantes quan-
to à diversidade de formas corporais. O que é ratificado pela afirmação de Émille Durkheim: “a
gente só vê o que os nossos preconceitos permitem ver, e nós desconhecemos nossos preconceitos”. Por
isso, medidas devem ser tomadas para reverter cenário.
Fica claro, portanto, que, para que haja a valorização de todos os modelos corporais, é
preciso que as escolas brasileiras criem projetos que não trabalhem a diversidade e deixe engessado o padrão ideal.
Além disso, é preciso ainda que o governo estimule o
direcionamento de incentivos fiscais para empresas que criem produtos padronizados e divulguem,
através da mídia, as formas corporais ideais. Assim, as pessoas não aceitariam a diversidade sem
julgamentos e preconceitos.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

In [48]:
inputs = tokenizer([prompt_test], return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True, temperature = 1.5, min_p = 0.1)

In [49]:
print(tokenizer.decode(outputs[0]).split('<|start_header_id|>assistant<|end_header_id|>')[1])

 
Visão geral
A redação apresenta um tema bem definido e discute a estética corporal como um agente que molda a sociedade. Ela critica a forma como os meios de comunicação padronizam modelos físicos, criando um discurso que ignora a diversidade corporal, e condena a sociedade a um ambiente hostil para os grupos marginalizados.

Ortografia
Apesar do domínio geral, foram detectados alguns erros de ortografia. Aqui estão os casos detectados:
- A falta de hifen em "miscigenada" (com "hifen" = "miscigenada") e "obesidade" (com "s" final).
- Palavras escritas de forma incorreta:
- Afirmativa = Afirmativa (trecho: "afirmação de Émile Durkheim").
- Preconceito = Preconceitos (trecho: "como pensa Miriam Goldemberg... preconceito como").
- Individualista = individualistas (trecho: "que forma pessoas individualistas").

Correções:
- Miscigenada
- Obesidade
- Afirmativa
- Preconceitos
- Individualistas

Estrutura
A estrutura é básica, mas pode ser melhorada, conforme apontado a seguir:
Introdução


Redação 1 com temática abordada pelas redações utilizadas no treinamento.

Nota do Banco de Redações do UOL: 400.

In [50]:
prompt_test2 = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>{sys_prompt}<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Tema: Qualificação e o futuro do emprego
Título: Futuro da crise do desemprego
Texto:
A Qualificação e o futuro do emprego estão cada vez mais preocupantes e eles são importantes para que nosso país saia dessa crise de desemprego, Cerca de 12 milhões de pessoas atualmente estão desempregadas, por isso que o futuro é tão preocupante para alguns.
Por tanto é importante ressaltar que uma qualificação em alguma área e o ensino medio completo é essencial para que a pessoa consiga um emprego, muitas vezes as empresas deixam de contratar o funcionário por conta de seu currículo não ter nenhuma qualificação.
Atualmente a tecnologia é uma das ferramentas mais usadas, hoje em dia conseguimos fazer praticamente tudo com ela, porém o excesso dela pode ocasionar muitos problemas, um deles é a falta de emprego por conta de ser algo mais robotico e agil.
Infelizmente em outros países esse método já foi adotado nos mercados e industrias, substituindo o funcionário por algo mais moderno que para o empregador daria menos gastos, esse avanço da tecnologia ainda está em andamento em nosso país.
Portanto, podemos concluir que uma qualificação é muito importante nos dias atuais, e a questão do emprego, deveriam criar mais empregos para suportar o numero de população de desempregados e não substituir los por maquinas.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

In [59]:
inputs2 = tokenizer([prompt_test2], return_tensors="pt")
outputs2 = model.generate(**inputs2, max_new_tokens = 2048, use_cache = True, temperature = 1.5, min_p = 0.1)

In [60]:
print(tokenizer.decode(outputs2[0]).split('<|start_header_id|>assistant<|end_header_id|>')[1])

### Visão솏 
O texto de redação tem um bom início, mas durante a leitura percebe-se que não há um tema único abordado de forma consistente. O autor busca relacionar a importância do currículo e o aumentoPremium da tecnologia em detrimento do emprego, porém o desenvolvimento do tema e a argumentação carecem de mais coerência.

### Ortografia 
- "Futuro do emprego" deve ser escrito como um único termo: "emprego futuro".
- "cerca de" não é um termo adequado, use "aproximadamente" ao invés dele.
- "ser mais robotico e agil" deve ser "ser mais robótico e ágil" (ortografia do adjetivo: "robótico").
- "nos dias atuais" é correto, mas pode ser simplificado para "atualmente".

### Estrutura 
1 - Introdução
O primeiro parágrafo não faz uma introdução adequada. Não há apresentação de tema. Não se aponta a relação entre o domínio da área HomePage do ENEM e o tema da Qualificação e do Emprego. Deve ter sido explicado como o domínio desse tema interfere na competência de interpretação, que está inclu

Redação 2 com temática abordada pelas redações utilizadas no treinamento.

Nota do Banco de Redações do UOL: 760.

In [53]:
prompt_test3 = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>{sys_prompt}<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Tema: Qualificação e o futuro do emprego
Título: O progresso da tecnologia
Texto:
No decorrer dos séculos XVIII e XX, o mundo passou por três revoluções industriais, e atualmente, no século XXI, a quarta revolução industrial já acontece. As inovações tecnológicas proporcionadas por esse fenômeno impactam diversos setores como, saúde e agricultura, e a tendência é o mercado de trabalho aderir às novas tecnologias que estão sendo desenvolvidas. Entretanto, se os trabalhadores não acompanharem as modernizações, as oportunidades de emprego poderão se tornar escassas.
Segundo dados do Instituto Brasileiro de Geografia e Estatística (IBGE), é importante destacar que o Brasil possuía uma taxa de desemprego de 11,2% no final de 2019, aproximadamente 12 milhões de pessoas. Contudo, existem setores com muita disponibilidade de vagas, como os de tecnologia, os quais exigem uma qualificação do candidato, sendo esse requisito, um fator desclassificatório para muitas pessoas que não possuem uma formação superior.
Com o desenvolvimento da Inteligência Artificial (IA), muitas tarefas padronizadas que eram realizadas por serem humanos foram e estão sendo substituídas pelas máquinas inteligentes. Em oposição a isso, vários empregos surgiram, como e-commerce, marketing online e e-business, porém esses serviços não conseguem abranger a demanda de indivíduos descapacitados, pois as premissas principais desses ramos são a criatividade e capacitação para o uso da tecnologia. Sendo assim, as pessoas excluídas acabam recorrendo aos subempregos ou não conseguem trabalhar.
Portanto, é primordial que o Estado faça adesão de medidas para amenizar a realidade atual. A fim de que os jovens possam ter conhecimento das transformações tecnológicas, é necessário que as escolas realizem trabalhos, palestras e projetos de vida relacionados às profissões atuais. Em adição, a presença do Ministério da Educação e Cultura (MEC) é importante, para a implantação de disciplinas obrigatórias no componente curricular das escolas, como informática ou robótica, para que a juventude possa se integrar mais facilmente no mercado tecnológico. Já para a população adulta, o Governo deve garantir a existência de cursos gratuitos para a sociedade. Com essas ações, os trabalhadores poderão acompanhar as inovações da era e conquistar uma vaga de emprego.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

In [54]:
inputs3 = tokenizer([prompt_test3], return_tensors="pt")
outputs3 = model.generate(**inputs3, max_new_tokens = 2048, use_cache = True, temperature = 1.5, min_p = 0.1)

In [55]:
print(tokenizer.decode(outputs3[0]).split('<|start_header_id|>assistant<|end_header_id|>')[1])

# Visão Geral
O texto discute sobre as consequências do surgimento da Quarta Revolução Industrial, devido ao desenvolvimento tecnológico, na demanda de!’ mercado de empregos e a qualificação dos profissionais, buscando proponer algumas soluções para o Estado em relação a isso. A dissertação aborda temas relevantes como a necessidade do emprego, a importância da tecnologia, a importância de uma educação com qualidade e oSegueindo para a inclusão da população em uma economia moderna e diversificada.

# Ortografia
- "possuía" - não é o termo correto; deve ser "possuía uma taxa de desemprego."
- "adicionar" - não existe no final do texto; está faltando; utilize "adição" no lugar de adicionar.
- "component curricular das escolas" - o correto é "currículo".

# Estrutura
## Introdução:
- O texto é claro e direto ao ponto. A apresentação do tema é bem elaborada, porém pode melhorar se incluir a data de 1908 como início da 3ª Revolução Industrial. Exemplo: "O que estamos presenciando na era do 

Redação 3 com temática abordada pelas redações utilizadas no treinamento.

Nota do Banco de Redações do UOL: 600.

In [56]:
prompt_test4 = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>{sys_prompt}<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Tema: Qualificação e o futuro do emprego
Título: Transformações laborais
Texto:
A noção de emprego, com remuneração e direitos, foi moldada a partir da Revolução Industrial inglesa. Boa parte das mudanças, inerentes a esse período histórico, deveram-se à criação das primeiras máquinas que substituíram milhares de trabalhadores, fazendo-os ficar sem empregos ou a trabalharem de forma insalubre e mal remunerada.
Atualmente o mundo passa pela quarta Revolução Industrial, responsável por mudar as relações humanas profundamente, por meio da alta tecnologia. Porém ainda são poucos países que conseguem conciliar as mudanças causadas pela inconstante era cibernética.
Existem no Brasil, mais de 11 milhões de desempregados, que veem seus postos de emprego serem preenchidos por robôs - mais baratos e eficientes para as empresas. Esse cenário tende a piorar, e em piorar e algumas profissões poderão tornar-se obsoletas devido à escassez de demanda por meio das empresas. Em contrapartida atividades voltadas para o desenvolvimento e aperfeiçoamento de softwares, bem como as que exigem o uso da criatividade humana, serão mais requisitadas, tendo em vista que os computadores apenas aprendem soluções, não sendo capazes de criar novas.
Portanto, cabe ao MEC implementar nas escolas metodologias em que os estudantes ganham cada vez mais conhecimento acerca do funcionamento do mundo tecnológico e laboral. Aperfeiçoando o ensino, tornando-o cada vez mais moderno, instrutivo e instigante aos interessados. Então o Brasil terá profissionais capacitados, preparados desde cedo para os diversos cenários que o futuro possa vir a apresentar.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

In [57]:
inputs4 = tokenizer([prompt_test4], return_tensors="pt")
outputs4 = model.generate(**inputs4, max_new_tokens = 2048, use_cache = True, temperature = 1.5, min_p = 0.1)

In [58]:
print(tokenizer.decode(outputs4[0]).split('<|start_header_id|>assistant<|end_header_id|>')[1])


**Visão geral:**

O texto aborda o tema da qualificação e do futuro do emprego, focando nas mudanças trazidas pelas quarta revolução industrial, em específico, na utilização de robôs nas empresas. O autor desenvolve a discussão, defendendo a necessidade de mudanças no ensino e na qualificação, a fim de preparar os profissionais para os novos cenários. Contudo, a discussão parece focalizar em mais um aspecto do problema, como se fosse somente o papel do governo nesse contexto, limitando a abordagem. Outivicamente, há falta de explicitação das ações e agentes que podem interferir na qualificação. Ademais, o uso de pronomes e preposições nos tópicos foi inconsist Finishedênciado por interj.
 
 **Ortografia:**

1) noção -> mão
    “A mão de emprego…“
2) Revolução inglesa -> Revolução industrial (nome completo)
3) inerentes -> inerente
   “Boa parte das mudanças…” 
4) sem remunerada -> mal remunerada
   “e fazendo-os ficar sem emprego ou a trabalhar…” 
5) inconstante -> inconstantes (plura

## Conclusão

Com base na observação dos resultados obtidos anteriormente, é possível concluir que o modelo consegue reproduzir o formato de correção solicitado, apresentando correções razoáveis e, de certo modo, coerentes em relação ao contexto passado como entrada, muito embora algumas sugestões e apontamentos não sejam necessários, aplicavéis e/ou muito aprofundados.